## Analisis de audio en Python


## 1. Cargar audio estereo y trabajar con el

Cargaremos un audio estereo descargado del campues para primeramente mostrar sus caracteristicas principales solicitadas en el PDF y luego lo reproducideremos

### 1.1. Cargar el audio estereo y mostrar sus caracteristicas


In [ ]:
#Importamos las librerias necesarias
import librosa  
import os  
import sys

# Cargar el archivo de audio en estéreo
audio_path = "interstellar.mp3"
audio_stereo, sr = librosa.load(audio_path, sr=None, mono=False)

# Mostrar características básicas
print("Frecuencia de muestreo:", sr)  # Muestras por segundo
print("Forma del array de audio:", audio_stereo.shape)  # Mostraea (canales, muestras) si el audio que pasamos es estereo

# Calcular tamaño del archivo
file_size = os.path.getsize(audio_path) / (1024 * 1024)  # Convertir bytes a MB
print("Tamaño del archivo: {:.2f} MB".format(file_size))

### 1.2. Reproducir audio estereo

In [ ]:
from IPython.display import Audio

# Reproducimos nuestro audio:
Audio(audio_path)




## 2. Convertimos audio estereo a mono y trabajamos con el

Comvertiremos el audio estereo a mono y también mostraremos sus caracteristicas y lo reproduciremos como antes hemos hecho con el audio estereo

### 2.1. Convertimos audio estereo a mono y mostramos caracteristicas

In [ ]:

# Convertir a mono 
audio_mono = librosa.to_mono(audio_stereo)

# Mostrar características
print("Frecuencia de muestreo:", sr)
print("Forma del array mono:", audio_mono.shape)

# Tamaño estimado del archivo
size_mono = sys.getsizeof(audio_mono) / (1024 * 1024)
print("Tamaño en memoria aproximado: {:.2f} MB".format(size_mono))


### 2.2. Reproducimos el audio

In [ ]:
import soundfile as sf
import sys
import numpy as np

max_val = np.abs(audio_mono).max() # Calculamos el valor absoluto maximo para prevenir errores al guardar el archivo por si algun valor se sale de rango
if max_val > 1.0:
    audio_mono = audio_mono / max_val # Si hay valores fuera de rango dividimos todos los valores por el maximo
audio_mono = audio_mono.astype(np.float32) # Convertimos a float32, formato adecuado para archivo .wav

# Guardar como archivo .wav
sf.write("interstellar.wav", audio_mono, sr)

# Reproducir con widget
Audio("interstellar.wav")


## 3. Mostramos grafica de audios

In [ ]:
%matplotlib inline

### 3.1. Mostramos grafica audio mono

In [ ]:
import matplotlib.pyplot as plt

muestras = sr * 5 # Calculamos cuantas muestras hay en 5 s
tiempo_mono = np.linspace(0, 5, muestras) #alineamos el eje x con el numero de muestras que vamos a graficas

plt.figure(figsize=(12, 4)) #Creamos una figura con dimensiones en pulgadas
plt.plot(tiempo_mono, audio_mono[:muestras], color='blue') #Dibujamos la forma de onda del audio mono
plt.title("Audio MONO - primeros 5 segundos")
plt.xlabel("Tiempo (s)")
plt.ylabel("Amplitud")
plt.grid(True) # Aniadimos una rejilla para facilirar la lectura de la grafica
plt.tight_layout() # Ajustamos bien los margenes
plt.show()




### 3.2. Mostramos grafica audio estereo

In [ ]:
tiempo_stereo = np.linspace(0, 5, muestras)

plt.figure(figsize=(12, 5))
plt.plot(tiempo_stereo, audio_stereo[0][:muestras], label="Canal izquierdo", alpha=0.7) #Dibujamos la forma de onda del canal izquierdo
plt.plot(tiempo_stereo, audio_stereo[1][:muestras], label="Canal derecho", alpha=0.7) # Dibujamos la forma de onda del canal derecho
plt.title("Audio ESTÉREO - primeros 5 segundos")
plt.xlabel("Tiempo (s)")
plt.ylabel("Amplitud")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

## 4. Diferencia entre audio estereo y mono

La principal diferencia entre estos dos tipos de audio podemos observarlo principalmente en la grafica, como vemos en el audio mono solo tenemos un unico canal de sonido es decir el sonido se esucha por igual por ambos altavoces, mientras que ene le audio estereo tenemos dos canales, el canal izquierdo, y el canal derecho, uno para el oido izquierdo y otro para el oido derecho.

## 5. Conceptos del audio digital

### 5.1. Frecuencia de muestreo
La frecuencia de muestro es la cantidad de muestras que toman por segundo para poder convertir las señales analógicas en señales digitales. Suele medirse en hercios(Hz).

### 5.2. Aliasing
Es un efecto no deseado que ocurre cuando la frecuencia de muestreo es demasiado baja, es decir, son indistinguibles al ser muestreadas digitalmente y por tanto no representa bien la señal original. Esto puede provocar confusión o distorsión en el sonido.

### 5.3. Profundidad de bits
La profundidad de bits es la cantidad de bits en una muestra, representando la amplitud de una onda. Cuantas más profundidad de bits, mayor número de divisiones verticales en los que se puede tomar la muestra.

### 5.4. Ancho de banda
El ancho de banda es la cantidad máxima de datos que se pueden transmitir a través de una conexión a Internet en un determinado tiempo. Se suele representar como el número de bits que se pueden transmitir en un segundo.

### 5.5. Tasa de bits
Podemos definir tasa de bits como el número de bits de datos que se envían cada segundo a través de un sistema de transmision digital o entre dos dispositivos digitales.

## 6. Aplicar la transformada rapida de Fourier (FFT)

In [ ]:
# Usamos 30 segundos del audio para crear la grafica
muestras = sr * 30 # Calculamos cuantas muestras hay en esos segundos
audio_fragmento = audio_mono[:muestras]

# Aplicamos la FFT 
fft = np.fft.fft(audio_fragmento)
magnitudes = np.abs(fft)  # Solo nos interesa la magnitud

# Creamos el eje de frecuencias correspondiente
frecuencias = np.fft.fftfreq(len(magnitudes), d=1/sr)

# Nos quedamos solo con la mitad positiva (la otra es simétrica)
half = len(magnitudes) // 2
frecuencias = frecuencias[:half]
magnitudes = magnitudes[:half]

# Graficamos
plt.figure(figsize=(12, 4))
plt.plot(frecuencias, magnitudes, color='purple')
plt.title("Dominio de la frecuencia (FFT del audio mono)")
plt.xlabel("Frecuencia (Hz)")
plt.ylabel("Magnitud")
plt.grid(True)
plt.tight_layout()
plt.show()


## 7. Calculo energia de espectograma y frecuencia de corte con epsilon

In [ ]:
audio_fragmento = audio_mono[:sr * 30]  # 30 segundos

# Calculamos el espectrograma que mostrara las frecuencias y como cambian en el tiempo
S = np.abs(librosa.stft(audio_fragmento)) ** 2  # potencia espectral

# Calculamos la energia media
espectro_promedio = np.mean(S, axis=1)

# Calculamos la energía total
energia_total = np.sum(espectro_promedio)

# Orden acumulado de energía
energia_acumulada = np.cumsum(espectro_promedio)
proporcion_acumulada = energia_acumulada / energia_total

# Elegimos un epsilon, en este caso el 95% de la energia
epsilon = 0.95

# Encontramos la frecuencia de corte
indice_corte = np.argmax(proporcion_acumulada >= epsilon)
frecuencia_corte = librosa.fft_frequencies(sr=sr)[indice_corte]

print(f"Frecuencia de corte para ε = {epsilon}: {frecuencia_corte:.2f} Hz")

#Mostramos grafica
frecuencias = librosa.fft_frequencies(sr=sr)

plt.figure(figsize=(12, 4))
plt.plot(frecuencias, espectro_promedio)
plt.axvline(frecuencia_corte, color='red', linestyle='--', label=f'ε = {epsilon} → {frecuencia_corte:.1f} Hz')
plt.title("Espectro promedio y frecuencia de corte")
plt.xlabel("Frecuencia (Hz)")
plt.ylabel("Energía")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

## 8. Aplicamos 'downsampling'

In [ ]:
# Calculamos la nueva frecuencia de muestreo
nuevo_sr = int(2 * frecuencia_corte)

# Aplicamos downsampling
audio_downsampled = librosa.resample(audio_mono, orig_sr=sr, target_sr=nuevo_sr)

# Guardamos el audio comprimido
sf.write("audio_downsampled.wav", audio_downsampled, nuevo_sr)

# Mostramos los nuevos datos
print(f"Nueva frecuencia de muestreo: {nuevo_sr} Hz")
print(f"Número de muestras original: {len(audio_mono)}")
print(f"Número de muestras tras downsampling: {len(audio_downsampled)}")


## 9. Espectogramas de archivo original y comprimido

In [ ]:


duracion = 30  # duracion del audio

# Recortamos ambos audios para el mismo tiempo
audio_original = audio_mono[:sr * duracion]
audio_comprimido = audio_downsampled[:nuevo_sr * duracion]

# Crear espectrogramas
S_orig = librosa.amplitude_to_db(np.abs(librosa.stft(audio_original)), ref=np.max)
S_comp = librosa.amplitude_to_db(np.abs(librosa.stft(audio_comprimido)), ref=np.max)

# Mostrar espectrogramas
plt.figure(figsize=(14, 6))

# Espectrograma original
plt.subplot(1, 2, 1)
librosa.display.specshow(S_orig, sr=sr, x_axis='time', y_axis='hz')
plt.title("Espectrograma - Audio original")
plt.colorbar(format="%+2.f dB")

# Espectrograma comprimido
plt.subplot(1, 2, 2)
librosa.display.specshow(S_comp, sr=nuevo_sr, x_axis='time', y_axis='hz')
plt.title("Espectrograma - Audio comprimido")
plt.colorbar(format="%+2.f dB")

plt.tight_layout()
plt.show()


## 9. Tamanio de ambos archivos

In [ ]:
sf.write("audio_original.wav", audio_mono, sr) # Alamcenamos el audio originar en un archivo

# Almacenamos los nombres de los archivos en variables
archivo_original = "audio_original.wav"
archivo_comprimido = "audio_downsampled.wav"

# Calculamos los tamanios en MB de cada archivo
tamano_original = os.path.getsize(archivo_original) / (1024 * 1024)
tamano_comprimido = os.path.getsize(archivo_comprimido) / (1024 * 1024)

print(f"Tamaño del archivo original: {tamano_original:.2f} MB")
print(f"Tamaño del archivo comprimido: {tamano_comprimido:.2f} MB")


## 10. Widgets de audios

### 10.1. Widget audio original

In [ ]:
Audio(archivo_original)


### 10.1. Widget audio comprimido

In [ ]:
Audio(archivo_comprimido)
